# Creating the Web Applications

In this section, we will focus on the specific aspects of creating web applications that are relevant to this project, rather than providing a detailed tutorial. The steps for creating and connecting web applications to the servers are similar for both components, so we will concentrate on the data component as an example.

## Web Application for the Data Component

The web application for the data component is designed to allow users to upload a CSV file for training and testing purposes. We have implemented a simple web application using Flask and HTML to achieve this. Below is the code for the application:

```python
from flask import Flask, request, redirect, url_for
import os

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = '.' 
app.config['ALLOWED_EXTENSIONS'] = {'csv'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

@app.route('/')
def index():
    return '''
    <!doctype html>
    <title>Upload CSV File</title>
    <h1>Upload CSV File</h1>
    <form action="/upload" method="post" enctype="multipart/form-data">
      <input type="file" name="file">
      <input type="submit" value="Upload">
    </form>
    '''

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        return redirect(request.url)
    if file and allowed_file(file.filename):
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], 'uploaded_file.csv')
        file.save(file_path)
        return redirect(url_for('index'))
    return redirect(request.url)

def run_app():
    app.run(host='0.0.0.0', port=8062)
``` 
## Key Points of the Web Application
1. **File Handling and Server Interaction**: The application does not communicate directly with the server. Instead, it saves the uploaded file locally, allowing the server to access it directly from the shared directory.

2. **Port Configuration**: The application runs on port 8062, which is required by the AI on Demand platform. If the application is not on this port, it will not be recognized during deployment.

3. **Application Start Method**: The application is not started within this file; instead, a function (run_app) is defined to start it. This approach aligns with the requirements of containerization, where both the server and the web application must be started from the same file.

These considerations are essential when creating web applications for this project, though there is flexibility in how you choose to implement the rest.

## starting the web application

The web application and server need to run concurrently. This is done through threading. Threading allows multiple sequences of instructions, or threads, to run concurrently within a single process. This means that both the server and a web application can run concurrently, allowing them to operate independently without blocking each other. In Python, this can be achieved using the threading module, where one thread starts the server and another launches the web application, ensuring both components run simultaneously within the same process. 

This means we will need to import the threading module into the server file, and start the application in a separate thread. The following code snippet demonstrates how to implement this in the server file:

```python
import threading
from app import run_app

# the rest of the code 

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    model_pb2_grpc.add_DataServiceServicer_to_server(DataServiceServicer(), server)
    port = 8061
    server.add_insecure_port('[::]:{}'.format(port))
    server.start()
    logging.info("Data service server started on port 8061.")
    threading.Thread(target=run_app).start()
    server.wait_for_termination()

if __name__ == '__main__':
    serve()
```

In this implementation, the web application is started in a separate thread with the following line:

```python
threading.Thread(target=run_app).start()
```

This ensures that when the serve function is called, both the server and the web application are initiated.

To test the components with their respective web applications, you can run the server as before:

```python
python data_service_server.py
```
When the server runs, you should see terminal output indicating that both the Flask application and the server have started. The Flask app's address will be displayed, allowing you to visit it and test the application.

You can find the web application for the test server in the directory 5. Creating the web applications\example\testing in the project repository. If you want another example on how to combine the application with the server you can take a look at it. With the necessary code for all components in place, the next step is to proceed with containerization.


